# Top-K Similarity Search - Ask A Book A Question

In this tutorial we will see a simple example of basic retrieval via Top-K Similarity search

In [1]:
# pip install langchain --upgrade
# Version: 0.0.164

# !pip install pypdf

In [1]:
# PDF Loaders. If unstructured gives you a hard time, try PyPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os

load_dotenv()

c:\Users\anand\Desktop\ananda github\Langchain-tutorial\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Load your data

Next let's load up some data. I've put a few 'loaders' on there which will load data from different locations. Feel free to use the one that suits you. The default one queries one of Paul Graham's essays for a simple example. This process will only stage the loader, not actually load it.

Then let's go ahead and actually load the data.

In [2]:
loader = PyPDFLoader("Net Centric Computing chapter 1 solutions.pdf")

In [3]:
data = loader.load()

Then let's actually check out what's been loaded

In [4]:
# Note: If you're using PyPDFLoader then it will split by page for you already
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your sample document')
print (f'Here is a sample: {data[0].page_content[:200]}')

You have 33 document(s) in your data
There are 1080 characters in your sample document
Here is a sample: Net Centric Computing (dot net) 
Old questions Solutions Chapter 1 
BSC CSIT 
1. Create class to showcase constructor, properties, indexers and encapsulation behavior of 
object-oriented language. [mo


### Chunk your data up into smaller documents

While we could pass the entire essay to a model w/ long context, we want to be picky about which information we share with our model. The better signal to noise ratio we have the more likely we are to get the right answer.

The first thing we'll do is chunk up our document into smaller pieces. The goal will be to take only a few of those smaller pieces and pass them to the LLM.

In [5]:
# We'll split our data into chunks around 500 characters each with a 50 character overlap. These are relatively small.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(data)

In [6]:
# Let's see how many small chunks we have
print (f'Now you have {len(texts)} documents')

Now you have 99 documents


### Create embeddings of your documents to get ready for semantic search

Next up we need to prepare for similarity searches. The way we do this is through embedding our documents (getting a vector per document).

This will help us compare documents later on.

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

C:\Users\anand\AppData\Local\Temp\ipykernel_71932\30168171.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Check to see if there is an environment variable with you API keys, if not, use what you put below

### Option #1: Chroma (for local)

I like Chroma becauase it's local and easy to set up without an account.

First we'll pass our texts to Chroma via `.from_documents`, this will 1) embed the documents and get a vector, then 2) add them to the vectorstore for retrieval later.

In [8]:
# load it into Chroma
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(texts, embedding)

Let's test it out. I want to see which documents are most closely related to a query.



In [27]:
query = "What peter thiel thinks about startups?"
docs = vectorstore.similarity_search(query)

Then we can check them out. In theory, the texts which are deemed most similar should hold the answer to our question.
But keep in mind that our query just happens to be a question, it could be a random statement or sentence and it would still work.

In [29]:
# Here's an example of the first document that was returned
for doc in docs:
    print (f"{doc.page_content}\n")

national security and global finance. He has provided early funding for LinkedIn, Yelp, and dozens of
successful technology startups, many run by former colleagues who have been dubbed the “PayPal
Mafia.” He is a partner at Founders Fund, a Silicon Valley venture capital firm that has funded
companies like SpaceX and Airbnb. He started the Thiel Fellowship, which ignited a national debate
by encouraging young people to put learning before schooling, and he leads the Thiel Foundation,

About the Authors
Peter Thiel
 is an entrepreneur and investor. He started PayPal in 1998, led it as CEO, and took it public in
2002, defining a new era of fast and secure online commerce. In 2004 he made the first outside
investment in Facebook, where he serves as a director. The same year he launched Palantir
Technologies, a software company that harnesses computers to empower human analysts in fields like

but he could never create an entire industry. Startups operate on the principle that you need to 

### Option #2: Pinecone (for cloud)
If you want to use pinecone, run the code below, if not then skip over to Chroma below it. You must go to [Pinecone.io](https://www.pinecone.io/) and set up an account

In [12]:
# PINECONE_API_KEY = os.getenv('PINECONE_API_KEY', 'YourAPIKey')
# PINECONE_API_ENV = os.getenv('PINECONE_API_ENV', 'us-east1-gcp') # You may need to switch with your env

# # initialize pinecone
# pinecone.init(
#     api_key=PINECONE_API_KEY,  # find at app.pinecone.io
#     environment=PINECONE_API_ENV  # next to api key in console
# )
# index_name = "langchaintest" # put in the name of your pinecone index here

# docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

### Query those docs to get your answer back

Great, those are just the docs which should hold our answer. Now we can pass those to a LangChain chain to query the LLM.

We could do this manually, but a chain is a convenient helper for us.

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
    temperature=0.7)

In [10]:

chain = load_qa_chain(llm, chain_type="stuff")

C:\Users\anand\AppData\Local\Temp\ipykernel_71932\4041488970.py:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")


In [17]:
query = "give me the exampe of asynchronous programming?"
docs = vectorstore.similarity_search(query)
docs

[Document(metadata={'source': 'Net Centric Computing chapter 1 solutions.pdf', 'author': "I'M Adarsha", 'creationdate': '2025-10-21T18:01:58+05:45', 'moddate': '2025-10-21T18:01:58+05:45', 'total_pages': 33, 'page': 24, 'creator': 'Microsoft® Word LTSC', 'page_label': '25', 'producer': 'Microsoft® Word LTSC'}, page_content='It uses async and await keywords, which make the code non-blocking, faster, and more \nresponsive — especially useful in desktop apps, web servers, and network programming. \nNeed for Asynchronous Programming: \n1. Non-blocking execution: \nThe main thread (UI or main program) continues working while waiting for time-\nconsuming tasks to complete. \n2. Improved performance: \nIt helps make programs faster by using system resources efficiently. \n3. Better user experience:'),
 Document(metadata={'page_label': '25', 'source': 'Net Centric Computing chapter 1 solutions.pdf', 'creator': 'Microsoft® Word LTSC', 'moddate': '2025-10-21T18:01:58+05:45', 'producer': 'Microso

Awesome! We just went and queried an external data source!

In [18]:
chain.run(input_documents=docs, question=query)

'Here\'s an example of asynchronous programming from the provided context:\n\n```csharp\nusing System;\nusing System.Threading.Tasks;\n\nclass Program\n{\n    static async Task Main()\n    {\n        Console.WriteLine("Downloading started...");\n        await DownloadFileAsync();  // Runs asynchronously\n    }\n\n    static async Task DownloadFileAsync()\n    {\n        await Task.Delay(3000); // Simulates time-consuming task\n        Console.WriteLine("File downloaded.");\n    }\n}\n```\n\n**Explanation:**\n\nIn this example:\n\n*   The `Main()` method calls the `DownloadFileAsync()` method using the `await` keyword.\n*   `DownloadFileAsync()` simulates a time-consuming task (like downloading a file) by pausing for 3 seconds using `Task.Delay(3000)`.\n*   During this 3-second delay, the program does not freeze. The `await` keyword ensures that the main thread is not blocked, allowing it to remain responsive (even though in this simple console app, there isn\'t much else for the main t